In [1]:
import numpy as np
import random, math
import copy

Dimension = 2
qubits = 2
A = np.array([[3, 1], [-1, 2]])
b = np.array([-1, 5])
# x1 = q1 + 2q2 - 4q3

QM = np.zeros(((qubits+1)*Dimension, (qubits+1)*Dimension))
### Linear terms ###
for k in range(Dimension):
    for i in range(Dimension):
        cef1 = pow(2,2*qubits)*pow(A[k][i],2)
        cef2 = pow(2,qubits+1)*A[k][i]*b[k]
        po2 = (qubits+1)*i + qubits 
        QM[po2][po2] = QM[po2][po2] + cef1 + cef2
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*b[k]
            po1 = (qubits+1)*i + l
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            
### First quadratic term ### 
for k in range(Dimension):
    for i in range(Dimension):
        for l in range (qubits):
            qcef = pow(2, l+qubits+1)*pow(A[k][i],2)
            po3 = (qubits+1)*i + l
            po4 = (qubits+1)*i + qubits 
            QM[po3][po4] = QM[po3][po4] - qcef
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = (qubits+1)*i + l1
                po2 = (qubits+1)*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                
### Second quadratic term ###                 
for k in range(Dimension):
    for i in range(Dimension-1):
        for j in range(i+1,Dimension):
            qcef = pow(2, 2*qubits+1)*A[k][i]*A[k][j]
            po3 = (qubits+1)*i + qubits
            po4 = (qubits+1)*j + qubits
            QM[po3][po4] = QM[po3][po4] + qcef
            for l in range (qubits):
                qcef = pow(2, l+qubits+1)*A[k][i]*A[k][j]
                po5 = (qubits+1)*i + qubits 
                po6 = (qubits+1)*j + l
                QM[po5][po6] = QM[po5][po6] - qcef
                po7 = (qubits+1)*i + l
                po8 = (qubits+1)*j + qubits
                QM[po7][po8] = QM[po7][po8] - qcef
            for l1 in range(qubits):
                for l2 in range(qubits):  
                    qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j] 
                    po1 = (qubits+1)*i + l1
                    po2 = (qubits+1)*j + l2
                    QM[po1][po2] = QM[po1][po2] + qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)
print("\nMinimum energy is ",-np.dot(b,b))
print("\n")

# Print Python code for the run in D-Wave quantum processing unit
print("Running code for D-Wave:\n")
print("from dwave.system import DWaveSampler, EmbeddingComposite")
print("sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))\n")
print("linear = {", end = "")
for i in range((qubits+1)*Dimension-1): 
    linear = i + 1
    print ("('q",linear,"','q",linear,"'):",format(QM[i][i]),sep='', end = ", ")
print ("('q",(qubits+1)*Dimension,"','q",(qubits+1)*Dimension,"'):",format(QM[(qubits+1)*Dimension-1][(qubits+1)*Dimension-1]),"}", sep='')

print("\nquadratic = {", end = "")
for i in range((qubits+1)*Dimension-1):
    for j in range(i+1,(qubits+1)*Dimension):
        qdrt1 = i + 1
        qdrt2 = j + 1
        if i == (qubits+1)*Dimension-2 and j == (qubits+1)*Dimension-1:
            print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), "}", sep='')
        else:
            print ("('q",qdrt1,"','q",qdrt2,"'):",format(QM[i][j]), sep ='', end = ", ")

print("\nQ = dict(linear)")
print("Q.update(quadratic)\n")

qa_iter = 1000
print("sampleset = sampler_auto.sample_qubo(Q, num_reads=",qa_iter,")", sep = "")
print("print(sampleset)")  

# Matrix Q is
[[  26.   40.  -80.    2.    4.   -8.]
 [   0.   72. -160.    4.    8.  -16.]
 [   0.    0.   96.   -8.  -16.   32.]
 [   0.    0.    0.  -13.   20.  -40.]
 [   0.    0.    0.    0.  -16.  -80.]
 [   0.    0.    0.    0.    0.  152.]]

Minimum energy is  -26


Running code for D-Wave:

from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):26.0, ('q2','q2'):72.0, ('q3','q3'):96.0, ('q4','q4'):-13.0, ('q5','q5'):-16.0, ('q6','q6'):152.0}

quadratic = {('q1','q2'):40.0, ('q1','q3'):-80.0, ('q1','q4'):2.0, ('q1','q5'):4.0, ('q1','q6'):-8.0, ('q2','q3'):-160.0, ('q2','q4'):4.0, ('q2','q5'):8.0, ('q2','q6'):-16.0, ('q3','q4'):-8.0, ('q3','q5'):-16.0, ('q3','q6'):32.0, ('q4','q5'):20.0, ('q4','q6'):-40.0, ('q5','q6'):-80.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)


In [2]:
from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):26.0, ('q2','q2'):72.0, ('q3','q3'):96.0, ('q4','q4'):-13.0, ('q5','q5'):-16.0, ('q6','q6'):152.0}

quadratic = {('q1','q2'):40.0, ('q1','q3'):-80.0, ('q1','q4'):2.0, ('q1','q5'):4.0, ('q1','q6'):-8.0, ('q2','q3'):-160.0, ('q2','q4'):4.0, ('q2','q5'):8.0, ('q2','q6'):-16.0, ('q3','q4'):-8.0, ('q3','q5'):-16.0, ('q3','q6'):32.0, ('q4','q5'):20.0, ('q4','q6'):-40.0, ('q5','q6'):-80.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

   q1 q2 q3 q4 q5 q6 energy num_oc. chain_.
0   1  1  1  0  1  0  -26.0     172     0.0
1   1  1  1  1  0  0  -21.0     196     0.0
2   1  1  1  1  1  0  -21.0     150     0.0
3   0  1  1  0  1  0  -16.0      28     0.0
4   0  0  0  0  1  0  -16.0     116     0.0
5   0  1  1  1  1  0  -13.0      49     0.0
6   0  0  0  1  0  0  -13.0     187     0.0
7   0  1  1  1  0  0   -9.0      25     0.0
8   0  0  0  1  1  0   -9.0      47     0.0
9   1  1  1  0  0  0   -6.0      13     0.0
10  0  0  0  0  0  0    0.0      13     0.0
11  0  1  1  0  0  0    8.0       1     0.0
12  1  0  0  1  0  0   15.0       1     0.0
13  1  1  1  1  1  1   19.0       2     0.0
['BINARY', 14 rows, 1000 samples, 6 variables]
